<a href="https://colab.research.google.com/github/quirkyabhi/EEG-EMOTIONAL-ANALYSIS/blob/master/codes/CNNwithkfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras import backend as K
K.set_image_dim_ordering('tf')
import os
import tensorflow as tf
import numpy as np
import scipy.io
import time
import datetime
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, StratifiedKFold

from scipy.interpolate import griddata
from sklearn.preprocessing import scale
from functools import reduce
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense, Conv2D, MaxPooling2D, Conv1D, MaxPool1D
from keras.models import Sequential
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
# from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
# import plotly.graph_objs as go
# from matplotlib.pyplot import cm
# from keras.models import Model
import numpy as np

import keras
# import h5py
from keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# def reformatInput(data, labels):
#     indices = np.random.permutation(147200)

#     trainIndices = [indices[:int(147200*.8)]]
#     validIndices = [indices[int(147200*.8):]]

#     if data.ndim == 3:
#         return [(data[trainIndices], np.squeeze(labels[trainIndices]).astype(np.int32)),
#                 (data[validIndices], np.squeeze(labels[validIndices]).astype(np.int32))]
# #                 (data[testIndices], np.squeeze(labels[testIndices]).astype(np.int32))]
#     elif data.ndim == 5:
#         return [(data[:, trainIndices], np.squeeze(labels[trainIndices]).astype(np.int32)),
#                 (data[:, validIndices], np.squeeze(labels[validIndices]).astype(np.int32))]

In [0]:
 def make_matrix(df):
    mat=np.array(df[1,:])
    return df.values

In [0]:
df=pd.read_csv('drive/My Drive/EEG/PLV_final_dist2.csv',  header=None)
ldf=pd.read_csv('drive/My Drive/EEG/arousal_label_total.csv', header= None)
mat=make_matrix(df)
ldf=make_matrix(ldf)
ldf=np.asarray(ldf)
mat.shape
df.shape

In [0]:
 X_train=df.iloc[:,:-1]
 y_train=df.iloc[:,-1]

In [0]:
# for i in range(len(mat)):
#     finalmat.append(mat[i,:].reshape(-1,384))
# train=np.asarray(finalmat)
# train.shape
# finalmat[1]

In [0]:
# (X_train, y_train), (X_test, y_test) = reformatInput(train, ldf)
# # y_train=to_categorical(y_train)
# # y_test=to_categorical(y_test)
# X_train = X_train.astype(float).reshape(117760,32,32,1)
# X_test = X_test.astype(float).reshape(147200-117760,32,32,1)
X_train=df.astype(float)
y_train=np.squeeze(y_train.astype(np.int32))
y_train.shape
X_train.shape
# y_train = keras.utils.to_categorical(y_train)
y_train

In [0]:
from keras import backend as K
K.set_image_dim_ordering('tf')
def make_model():
  
  num_category = 2
  # t_train=y_train
  # y_test=y_val
  # y_train = keras.utils.to_categorical(y_train, num_category)
  # y_test = keras.utils.to_categorical(y_val, num_category)
  model = Sequential()
  #convolutional layer with rectified linear unit activation
  model.add(Conv2D(32, kernel_size=3,activation='relu',input_shape=(32,32,1), 
                   kernel_initializer=keras.initializers.RandomNormal(stddev=0.1), bias_initializer=keras.initializers.RandomNormal(value=0.1)))
  #32 convolution filters used each of size 3x3
  #again
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
#   model.add(Dropout(0.1))
  model.add(Conv2D(64, 3, activation='relu',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.1), bias_initializer=keras.initializers.Constant(value=0.1)))
  model.add(Conv2D(64, kernel_size=3,activation='relu',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.1), bias_initializer=keras.initializers.Constant(value=0.1)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  #64 convolution filters used each of size 3x3
  #choose the best features via pooling
#   model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())

  model.add(Conv2D(128, kernel_size=3,activation='relu',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.1), bias_initializer=keras.initializers.Constant(value=0.1)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  
#   model.add(Dropout(0.2))
  
#   model.add(BatchNormalization())
#   model.add(Dropout(0.2))
  # randomly turn neurons on and off to improve convergence
  # model.add(Dropout(0.25))
  # model.add(Conv2D(256, (3, 3), activation='relu'))
  # #64 convolution filters used each of size 3x3
  # #choose the best features via pooling
  # model.add(MaxPooling2D(pool_size=(2, 2)))
  # model.add(BatchNormalization())
  # model.add(Dropout(0.2))
  # model.add(Conv2D(512, kernel_size=(3, 3),
  #                  activation='relu'))
  # model.add(MaxPooling2D(pool_size=(2, 2)))
  # model.add(BatchNormalization())
  # model.add(Dropout(0.1))
  # # randomly turn neurons on and off to improve convergence
  # model.add(Dropout(0.25))
  # # flatten since too many dimensions, we only want a classification output
  model.add(Flatten())
  #fully connected to get all relevant data
  model.add(Dense(256, activation='relu'))
  #one more dropout for convergence' sake :) 
#   model.add(Dropout(0.2))
  #output a softmax to squash the matrix into output probabilities
  model.add(Dense(2, activation='softmax'))
  print(model.summary())
  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
  return model
# model=make_model()

In [0]:
# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])
# X_test=to_categorical(X_test)
# y_test=to_categorical(y_test)
# batch_size = 256
# num_epoch = 100
# model_log = model.fit(X_train, y_train,
#           batch_size=batch_size,
#           epochs=num_epoch,
#           verbose=1,
#           validation_data=(X_test, y_test))

In [0]:
# def make_model():
#   n_cols=X_train.shape[1]
#   model=Sequential()
#   model.add(Dense(664, kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None),
#                 bias_initializer=keras.initializers.Constant(value=0.1),activation='relu',input_shape=(n_cols,)))
#   model.add(Dense(664, kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None),
#                 bias_initializer=keras.initializers.Constant(value=0.1),activation='relu'))
#   model.add(Dense(664,activation='relu'))
#   model.add(Dense(664,activation='relu'))
#   model.add(Dense(332,activation='relu'))
#   model.add(Dense(166,activation='relu'))
#   model.add(Dense(83,activation='relu'))
#   model.add(Dense(42,activation='relu'))
#   model.add(Dense(21,activation='relu'))
#   model.add(Dense(10,activation='relu'))
#   model.add(Dense(4,activation='relu')) 
#   model.add(Dense(2,activation='softmax'))
#   model.compile(loss=keras.losses.sparse_categorical_crossentropy,
#               optimizer=keras.optimizers.Adam(.001),
#               metrics=['accuracy'])
#   return model

# model=make_model()
# model.summary()
  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_553 (Dense)            (None, 664)               442224    
_________________________________________________________________
dense_554 (Dense)            (None, 664)               441560    
_________________________________________________________________
dense_555 (Dense)            (None, 664)               441560    
_________________________________________________________________
dense_556 (Dense)            (None, 664)               441560    
_________________________________________________________________
dense_557 (Dense)            (None, 332)               220780    
_________________________________________________________________
dense_558 (Dense)            (None, 166)               55278     
_________________________________________________________________
dense_559 (Dense)            (None, 83)                13861     
__________

In [0]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('cnn', KerasClassifier(build_fn=make_model, epochs=50, batch_size=256, verbose=1, shuffle=True)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(pipeline, X_train,y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/50
1023/1023 [==============================] - 7s 7ms/step - loss: 0.6957 - acc: 0.5494
Epoch 2/50
1023/1023 [==============================] - 0s 49us/step - loss: 0.6469 - acc: 0.5894
Epoch 3/50
1023/1023 [==============================] - 0s 51us/step - loss: 0.5869 - acc: 0.6305
Epoch 4/50
1023/1023 [==============================] - 0s 49us/step - loss: 0.5288 - acc: 0.7576
Epoch 5/50
1023/1023 [==============================] - 0s 51us/step - loss: 0.3810 - acc: 0.8563
Epoch 6/50
1023/1023 [==============================] - 0s 49us/step - loss: 0.2861 - acc: 0.8866
Epoch 7/50
1023/1023 [==============================] - 0s 49us/step - loss: 0.2143 - acc: 0.9013
Epoch 8/50
1023/1023 [==============================] - 0s 48us/step - loss: 0.1489 - acc: 0.9316
Epoch 9/50
1023/1023 [==============================] - 0s 60us/step - loss: 0.0930 - acc: 0.9462
Epoch 10/50
1023/1023 [==============================] - 0s 53us/step - loss: 0.0962 - acc: 0.9541
Epoch 11/50
1023/102

In [0]:
results

array([0.94552529, 0.58984375, 0.9453125 , 0.8828125 , 0.92941177])